In [21]:
from datetime import datetime
from mypackage.helpers import Helper
from mypackage.constants import COLOR_DICT, SORTER, KK_DICT, FLAVOUR_DICT, SHOP_NAME
import io

In [2]:
import os
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaFileUpload, MediaIoBaseDownload

from httplib2 import Http
import pandas as pd
from decouple import config

from mypackage.helpers import Helper
from mypackage.constants import SHOP_NAME

#setup
CLIENT_FILE = config("CLIENT_FILE")
SCOPES = ["https://www.googleapis.com/auth/forms.body", "https://www.googleapis.com/auth/drive", "https://www.googleapis.com/auth/drive.file", "https://www.googleapis.com/auth/forms.responses.readonly", "https://www.googleapis.com/auth/spreadsheets.readonly"]
DISCOVERY_DOC = "https://forms.googleapis.com/$discovery/rest?version=v1"

creds = None

if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(CLIENT_FILE, SCOPES)
        creds = flow.run_local_server(port=0)
    with open('token.json', 'w') as token:
        token.write(creds.to_json())

kimseng_helper_obj = Helper.KimSeng()
jackson_helper_obj = Helper.Jackson()

today_str = datetime.today().strftime('%d-%m-%Y')

In [40]:
service = build('forms', 'v1', credentials=creds) #forms
service_sheet = build('sheets', 'v4', credentials=creds)
service_drive = build('drive', 'v3', credentials=creds)
GOOGLE_SHEET_ID = '1ey4vDFNm3qIogHpLeUVkABsgIbD0UskUAdTnvuentng'
# FORMS_API_KEY = config("FORMS_API_KEY")
# service = build('forms', 'v1', credentials=creds, developerKey=FORMS_API_KEY, discoveryServiceUrl=DISCOVERY_DOC)
#     discoveryServiceUrl=f'https://forms.googleapis.com/$discovery/rest?version=v1beta&key={FORMS_API_KEY}&labels=FORMS_BETA_TESTERS',
#     num_retries=3
# )

def create_excel_input(formId):
    '''
    Retrieve google sheet and create csv locally
    '''
    # Retrieve google sheet
    form = service.forms().get(formId=formId).execute()
    sheet_res = service_sheet.spreadsheets().get(spreadsheetId=form['linkedSheetId']).execute()
    dataset = service_sheet.spreadsheets().values().get(
        spreadsheetId= sheet_res['spreadsheetId'],
        range=sheet_res['sheets'][0]['properties']['title'],
        majorDimension= 'ROWS',
    ).execute()
    df = pd.DataFrame(dataset['values'])
    df = df.rename(columns=df.iloc[0]).drop(df.index[0])

    # shop_name = SHOP_NAME['JS']
    # shop_input_path = os.path.join(shop_name, 'input')
    # final_path = os.path.join(shop_input_path, f"{sheet_res['properties']['title'].replace('/', '-')}_Generated.csv")
    file_name = f"{sheet_res['properties']['title'].replace('/', '-')}_Generated.csv"
    df.to_csv(file_name, index=False)
    return df, file_name

def create_all_output(df_input, is_kimseng):
    if is_kimseng:
        excel_output_path, image_output_path, txt_name = kimseng_helper_obj.run_ks(df_input, today_str)
        return excel_output_path, image_output_path, txt_name
    else:
        excel_output_path, image_output_path, txt_name = jackson_helper_obj.run_js(df_input, today_str)
        return excel_output_path, image_output_path, txt_name

In [4]:
#Search or Create folders in google drive
def get_or_create_folder_id(file_name, parents=None):
    '''
    file_name: File's Name
    parents: [File Id]
    '''
    files = service_drive.files().list( q=f"name = '{file_name}' and mimeType = 'application/vnd.google-apps.folder' and trashed = false" ,
                                       spaces='drive',).execute()['files']
    if len(files) > 0: #exist
        return files[0].get('id')
    
    #create folder
    file_metadata = {
            'name': f"{file_name}",
            'mimeType': 'application/vnd.google-apps.folder',
            'parents': parents, 
        }

    res_file = service_drive.files().create(body=file_metadata, fields='id').execute()
    return res_file.get('id')

#create stuff
#so the idea here is to export to files, upload the files, and delete the files. can be triggered through the form id

ks_j_folder_id = get_or_create_folder_id('KS&J&HYK')
kimseng_folder_id = get_or_create_folder_id('KimSeng', [ks_j_folder_id])
kimseng_folder_input_id = get_or_create_folder_id('KimSengInput', [kimseng_folder_id])
kimseng_folder_output_id = get_or_create_folder_id('KimSengOutput', [kimseng_folder_id])
kimseng_folder_output_today_id = get_or_create_folder_id(f'{today_str} Koutput', [kimseng_folder_output_id])

# jackson_folder_id = get_or_create_folder_id('JDinner', [ks_j_folder_id])
# jackson_folder_input_id = get_or_create_folder_id('JDinnerInput', [jackson_folder_id])
# jackson_folder_output_id = get_or_create_folder_id('JDinnerOutput', [jackson_folder_id])
# jackson_folder_output_today_id = get_or_create_folder_id(f'{today_str} Joutput', [jackson_folder_output_id])

hyk_folder_id = get_or_create_folder_id('HYK', [ks_j_folder_id])
hyk_folder_input_id = get_or_create_folder_id('HYKInput', [hyk_folder_id])
hyk_folder_output_id = get_or_create_folder_id('HYKOutput', [hyk_folder_id])
hyk_folder_output_today_id = get_or_create_folder_id(f'{today_str} HYKOutput', [hyk_folder_output_id])

#put csv into input
# f"{sheet_res['properties']['title'].replace('/', '-')}_Generated"

In [ ]:

def file_exist(file_name, mimetype):
    files = service_drive.files().list( q=f"name = '{file_name}' and mimeType = '{mimetype}' and trashed = false" ,
                                       spaces='drive',).execute()['files']
    if len(files) > 0: #exist
        return files[0].get('id')
    return False

def upload_input_file(file_name, parents):
    '''
    Input:
    file_name: File's Name
    parents: [File Id]
    '''
    file_metadata = {'name': file_name,'parents': parents,}
    file_media = MediaFileUpload(file_name, mimetype='text/csv')
    if file_id := file_exist(file_name, 'text/csv'):
        service_drive.files().update(fileId=file_id, body={'name': file_name}, media_body=file_media, fields='id').execute()['id']
    else:
        file_id = service_drive.files().create(body=file_metadata, media_body=file_media, fields='id').execute()['id']
    file_media = None #To stop reading the file to allow delete

def upload_output_file(excel_name, image_name, text_name, parents):
    '''
    Output
    file_name: File's Name
    parents: [File Id]
    '''


    image_metadata = {'name': image_name, 'parents': parents,}
    image_media = MediaFileUpload(image_name, mimetype='image/png')
    if image_file_id := file_exist(image_name, 'image/png'):
        service_drive.files().update(fileId= image_file_id, body={'name': image_name}, media_body=image_media, fields='id').execute()['id']
    else:
        image_file_id = service_drive.files().create(body=image_metadata, media_body=image_media, fields='id').execute()['id']
    image_media=None

    text_metadata = {'name': text_name,'parents': parents,}
    text_media = MediaFileUpload(text_name, mimetype='text/plain')
    if text_file_id := file_exist(text_name, 'text/plain'):
        service_drive.files().update(fileId=text_file_id, body={'name': text_name}, media_body=text_media, fields='id').execute()['id']
    else:
        text_file_id = service_drive.files().create(body=text_metadata, media_body=text_media, fields='id').execute()['id']
    text_media=None

    excel_metadata = {'name': excel_name,'parents': parents}
    excel_media = MediaFileUpload(excel_name, mimetype='application/vnd.openxmlformats-officedocument.spreadsheetml.sheet')
    if excel_file_id := file_exist(excel_name, 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'):
        service_drive.files().update(fileId=excel_file_id, body={'name': excel_name}, media_body=excel_media, fields='id').execute()['id']
    else:
        excel_file_id = service_drive.files().create(body=excel_metadata, media_body=excel_media, fields='id').execute()['id']
    excel_media=None

#get sheet list
sheet_res = service_sheet.spreadsheets().get(spreadsheetId=GOOGLE_SHEET_ID).execute()
dataset = service_sheet.spreadsheets().values().get(
        spreadsheetId= sheet_res['spreadsheetId'],
        range=sheet_res['sheets'][0]['properties']['title'],
        majorDimension= 'ROWS',
    ).execute()
google_sheet_df = pd.DataFrame(dataset['values'])
google_sheet_df = google_sheet_df.rename(columns=google_sheet_df.iloc[0]).drop(google_sheet_df.index[0])
google_sheet_df = google_sheet_df[google_sheet_df['Date'] == today_str]


#Kimseng
kimseng_sheet_id = google_sheet_df["KimSeng"].item()
df_input, excel_input_name = create_excel_input(kimseng_sheet_id)
excel_output_path, image_output_path, txt_name = create_all_output(df_input, True)
upload_input_file(excel_input_name, [kimseng_folder_input_id])
os.remove(excel_input_name)

upload_output_file(excel_output_path, image_output_path, txt_name, [kimseng_folder_output_today_id])
os.remove(excel_output_path)
os.remove(image_output_path)
os.remove(txt_name)

# #Jackson
# jackson_sheet_id = google_sheet_df["Jackson"].item()
# df_input, excel_input_name = create_excel_input(jackson_sheet_id)
# excel_output_path, image_output_path, txt_name = create_all_output(df_input, False)
# upload_input_file(excel_input_name, [jackson_folder_input_id])
# os.remove(excel_input_name)

# upload_output_file(excel_output_path, image_output_path, txt_name, [jackson_folder_output_today_id])
# os.remove(excel_output_path)
# os.remove(image_output_path)
# os.remove(txt_name)

#hyk
hyk_sheet_id = google_sheet_df["HYK"].item()
df_input, excel_input_name = create_excel_input(hyk_sheet_id)
excel_output_path, image_output_path, txt_name = create_all_output(df_input, True)
upload_input_file(excel_input_name, [hyk_folder_input_id])
os.remove(excel_input_name)

upload_output_file(excel_output_path, image_output_path, txt_name, [hyk_folder_output_today_id])
os.remove(excel_output_path)
os.remove(image_output_path)
os.remove(txt_name)

In [ ]:
# # Request body for creating a form
# NEW_FORM = {
#     "info": {
#         "title": "Quickstart form",
#     }
# }

# # Request body to add a multiple-choice question
# NEW_QUESTION = {
#     "requests": [{
#         "createItem": {
#             "item": {
#                 "title": "In what year did the United States land a mission on the moon?",
#                 "questionItem": {
#                     "question": {
#                         "required": True,
#                         "choiceQuestion": {
#                             "type": "RADIO",
#                             "options": [
#                                 {"value": "1965"},
#                                 {"value": "1967"},
#                                 {"value": "1969"},
#                                 {"value": "1971"}
#                             ],
#                             "shuffle": True
#                         }
#                     }
#                 },
#             },
#             "location": {
#                 "index": 0
#             }
#         }
#     }]
# }

# # Creates the initial form
# # result = form_service.forms().create(body=NEW_FORM).execute()

# # Adds the question to the form
# # question_setting = form_service.forms().batchUpdate(formId=result["formId"], body=NEW_QUESTION).execute()

# # Prints the result to show the question has been added
# get_result = form_service.forms().get(formId="1q9p8iXFZl-aSsxpeci4FhgWE2GIn_-xLuNI_yzhy0lY").execute()
# print(get_result)